<a href="https://colab.research.google.com/github/sathu0622/25-26J-438-AI-Powered-LMS-for-Visually-Impaired-Students/blob/AI-Powered-System-for-Voice-Based-Resource-Type-Summarization-of-Historical-Content-for-VIS/Summarization2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===================== 2️⃣ Mount Google Drive =====================
from google.colab import drive
drive.mount('/content/drive')

# ===================== 3️⃣ Load Dataset =====================
import json
from datasets import Dataset

dataset_path = "/content/drive/MyDrive/history_dataset.json"  # update path

with open(dataset_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

dataset = Dataset.from_list([{
    'text': item['content'],
    'summary': item['target_summary'],
    'source_type': item['source_type']
} for item in data])

dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset['train']
test_dataset = dataset['test']

print(f"Train samples: {len(train_dataset)}, Test samples: {len(test_dataset)}")

# ===================== 4️⃣ Tokenization =====================
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
max_input_length = 1024
max_target_length_map = {'newspaper': 120, 'magazine': 220, 'book': 400}

def preprocess(batch):
    max_target_lengths = [max_target_length_map.get(src, 150) for src in batch['source_type']]

    inputs = tokenizer(batch['text'], max_length=max_input_length, truncation=True, padding="max_length")
    targets = [tokenizer(batch['summary'][i], max_length=max_target_lengths[i], truncation=True, padding="max_length")
               for i in range(len(batch['summary']))]

    batch["input_ids"] = inputs["input_ids"]
    batch["attention_mask"] = inputs["attention_mask"]
    batch["labels"] = [t['input_ids'] for t in targets]
    return batch

train_dataset = train_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

# ===================== 5️⃣ Load Model =====================
from transformers import BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# ===================== 6️⃣ Check GPU =====================
import torch
use_fp16 = torch.cuda.is_available()
print(f"GPU available: {use_fp16}")

# ===================== 7️⃣ Training Arguments =====================
from transformers import TrainingArguments

output_dir = "/content/drive/MyDrive/history_summary_model"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    learning_rate=3e-5,
    weight_decay=0.01,
    fp16=use_fp16
)

# ===================== 8️⃣ Data Collator =====================
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ===================== 9️⃣ Trainer =====================
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# ===================== 🔟 Train =====================
trainer.train()

# ===================== 1️⃣1️⃣ Save Final Model =====================
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")

# ===================== 1️⃣2️⃣ Generate Summaries =====================
def summarize(text, source_type='magazine'):
    max_length = max_target_length_map.get(source_type, 150)
    inputs = tokenizer(text, return_tensors="pt", max_length=max_input_length, truncation=True)
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        min_length=int(max_length*0.5),
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage
example_text = train_dataset[0]['text']
example_type = train_dataset[0]['source_type']
print("Generated Summary:")
print(summarize(example_text, source_type=example_type))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train samples: 360, Test samples: 41


Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/41 [00:00<?, ? examples/s]

GPU available: True


/tmp/ipython-input-3014397194.py:84: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sathushan622 (sathushan622-sliit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,3.898700
20,2.549800
30,2.349400
40,2.085400
50,1.739100
60,1.841100
70,1.915200
80,1.925800
90,1.974800
100,1.945000


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Model saved to /content/drive/MyDrive/history_summary_model
Generated Summary:


RuntimeError: Expected all tensors to be on the same device, but got index is on cpu, different from other tensors on cuda:0 (when checking argument in method wrapper_CUDA__index_select)

In [ ]:
# ===================== 0️⃣ Install Dependencies =====================
!pip install -q evaluate rouge_score transformers

# ===================== 1️⃣ Load the Saved Model =====================
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

model_path = "/content/drive/MyDrive/history_summary_model"
tokenizer = BartTokenizer.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path)

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print("Model loaded. Using device:", device)

# ===================== 2️⃣ Import Evaluation Library =====================
import evaluate
rouge = evaluate.load("rouge")

# ===================== 3️⃣ Define Summarization Function =====================
max_target_length_map = {'newspaper': 120, 'magazine': 220, 'book': 400}

def summarize(text, source_type='magazine'):
    max_length = max_target_length_map.get(source_type, 150)
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True).to(device)
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        min_length=int(max_length*0.5),
        length_penalty=2.0,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# ===================== 4️⃣ Generate Summaries on Test Dataset =====================
generated_summaries = []
reference_summaries = []

for example in test_dataset:
    pred = summarize(example['text'], example['source_type'])
    generated_summaries.append(pred)
    reference_summaries.append(example['summary'])

# ===================== 5️⃣ Print Some Examples =====================
for i in range(5):
    print(f"--- Example {i+1} ---")
    print("Source Type:", test_dataset[i]['source_type'])
    print("Original Text:", test_dataset[i]['text'][:500], "...")
    print("Target Summary:", reference_summaries[i])
    print("Generated Summary:", generated_summaries[i])
    print("\n")

# ===================== 6️⃣ Calculate ROUGE Scores =====================
results = rouge.compute(predictions=generated_summaries, references=reference_summaries)
print("ROUGE Scores:")
for key in results:
    print(f"{key}: {results[key].mid.fmeasure:.4f}")


  Preparing metadata (setup.py) ... done
Model loaded. Using device: cuda


/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:1733: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


--- Example 1 ---
Source Type: magazine
Original Text: The Seventh Dragoon Guards made one of the last charges of the First World War at 10.30am on 11 November 1918, galloping forward and capturing the bridge at Lessines in the Picardy region of Belgium. Fighting officially ended at 11am on the same day, when the Armistice was signed. On hearing the news, the Fifth Dragoon Guards  official war diary described the celebration as the regiment prepared for a  triumphal march into Germany . While the official diary could easily be written in a celebrato ...
Target Summary: The First World War’s thunderous end arrived with a stark paradox on November 11, 1918. Even as the official Armistice was hours away, the Seventh Dragoon Guards made one of the conflict's last charges, capturing a bridge at Lessines. While official war diaries optimistically spoke of triumphal marches into Germany and newspapers at home erupted in jubilant celebration, the news of peace was often met with a far more com

AttributeError: 'numpy.float64' object has no attribute 'mid'